# データ収集

## 公式記録から勝敗データを収集

[公式記録](https://data.j-league.or.jp/)からデータを収集する。

APIとかで取得できるようにはなっていないので、HTMLをパースして加工しやすいような形式にすることを考える。

`https://data.j-league.or.jp/SFMS01/search?competition_years=<年度>&competition_frame_ids=<カテゴリ>`

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
year = 2018
competition = 2
url = 'https://data.j-league.or.jp/SFMS01/search?competition_years='+ str(year) + '&competition_frame_ids=' + str(competition)

print("URL= " + url)
resp = requests.get(url)
#print(resp.text)

URL= https://data.j-league.or.jp/SFMS01/search?competition_years=2018&competition_frame_ids=2


HTMLを直接見ると、tableタグのデータを見れば良さそう。

Tableタグの取得は、pandasで簡単に取れるらしい([参考](https://qiita.com/kitsuyui/items/4906bb457af4d0e2d0a5))ので、これを使う。

In [3]:
import pandas as pd
from IPython.core.display import display
dfs = pd.io.html.read_html(url)
dfs[0] = dfs[0].drop(['インターネット中継・TV放送','入場者数'],axis=1) # 結果に関係なさそうな放送局と入場者数はこの時点で削除しておく
display(dfs[0])

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム
0,2018,Ｊ２,第１節第１日,02/25(日),13:04,福岡,2-0,岐阜,レベスタ
1,2018,Ｊ２,第１節第１日,02/25(日),14:03,栃木,2-4,大分,栃木グ
2,2018,Ｊ２,第１節第１日,02/25(日),14:03,愛媛,1-2,金沢,ニンスタ
3,2018,Ｊ２,第１節第１日,02/25(日),14:03,東京Ｖ,2-1,千葉,味スタ
4,2018,Ｊ２,第１節第１日,02/25(日),14:03,讃岐,0-1,新潟,ピカスタ
5,2018,Ｊ２,第１節第１日,02/25(日),14:03,水戸,3-0,山形,Ｋｓスタ
6,2018,Ｊ２,第１節第１日,02/25(日),14:03,横浜FC,0-0,松本,ニッパツ
7,2018,Ｊ２,第１節第１日,02/25(日),14:05,徳島,0-1,岡山,鳴門大塚
8,2018,Ｊ２,第１節第１日,02/25(日),15:03,京都,0-2,町田,西京極
9,2018,Ｊ２,第１節第１日,02/25(日),15:34,山口,4-1,熊本,みらスタ


ここまでで、[download.py](download.py)の関数download_resultを作成したので、これで試してみる。

In [4]:
import download
df = download.download_result(year=1992)
display(df)

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,1992,ＹＮＣ,第１節第１日,09/05(土),15:00,鹿島,4-2,横浜Ｆ,笠松,5226,NaN
1,1992,ＹＮＣ,第１節第１日,09/05(土),15:30,名古屋,3-0,清水,瑞穂球,8029,NaN
2,1992,ＹＮＣ,第１節第１日,09/05(土),18:02,浦和,2-3,市原,大宮,4934,NaN
3,1992,ＹＮＣ,第１節第１日,09/05(土),19:00,Ｇ大阪,2-0,横浜M,神戸中央,4728,NaN
4,1992,ＹＮＣ,第１節第２日,09/06(日),19:00,広島,2-3,Ｖ川崎,広島ス,13861,NaN
5,1992,ＹＮＣ,第２節第１日,09/09(水),18:30,広島,2-3,浦和,広島ス,3128,NaN
6,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,鹿島,4-3,Ｖ川崎,国立,18009,NaN
7,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,清水,2-1,横浜M,日本平,8032,NaN
8,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,名古屋,2-1,横浜Ｆ,瑞穂球,5165,NaN
9,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,Ｇ大阪,1-0,市原,神戸中央,2760,NaN


CSVとかでいいけど、最近データベースを勉強中なのでsqlite3に保存してみる  
pandasのDataFrameはto_sqlメソッドでSQLに変換可能らしい([参考](http://www.stockdog.work/entry/2017/03/22/010057))。

保存場所として、data/jleague.dbとするものとした。  
この場合、dataフォルダが存在しない場合は作ることにする。

In [5]:
import sqlite3
import os
db_name = "data/test.db"
if os.path.exists('data') == False:
    os.mkdir('data')
conn = sqlite3.connect(db_name) # DB接続
dfs[0].to_sql("result", conn, if_exists='replace') # DFをDBに変換

ここまでで、download_all_resultsにした。

download_all_resultsは、1992年～2017年のデータを取得し、sqlite(data/jleague.db)に変換する。  
jleague.dbのresultというデータベースに、取得した結果を格納する。

In [6]:
download.download_all_results()

生成されたDBを見てみる。

In [7]:
import sqlite3
import pandas as pd
db_name = 'data/jleague.db'
conn = sqlite3.connect(db_name)
df = pd.read_sql("select * from result", conn)
conn.close()

display(df)

,index,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,0,1992,ＹＮＣ,第１節第１日,09/05(土),15:00,鹿島,4-2,横浜Ｆ,笠松,5226.0,None
1,1,1992,ＹＮＣ,第１節第１日,09/05(土),15:30,名古屋,3-0,清水,瑞穂球,8029.0,None
2,2,1992,ＹＮＣ,第１節第１日,09/05(土),18:02,浦和,2-3,市原,大宮,4934.0,None
3,3,1992,ＹＮＣ,第１節第１日,09/05(土),19:00,Ｇ大阪,2-0,横浜M,神戸中央,4728.0,None
4,4,1992,ＹＮＣ,第１節第２日,09/06(日),19:00,広島,2-3,Ｖ川崎,広島ス,13861.0,None
5,5,1992,ＹＮＣ,第２節第１日,09/09(水),18:30,広島,2-3,浦和,広島ス,3128.0,None
6,6,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,鹿島,4-3,Ｖ川崎,国立,18009.0,None
7,7,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,清水,2-1,横浜M,日本平,8032.0,None
8,8,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,名古屋,2-1,横浜Ｆ,瑞穂球,5165.0,None
9,9,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,Ｇ大阪,1-0,市原,神戸中央,2760.0,None


全18118レコード取得可能。

## Football-Labからデータを収集

[Football-Lab](http://www.football-lab.jp/)はJリーグのデータ分析で有名なデータスタジアムが運営するWebサイト。  
勝敗データだけでなく、得点がどんな得点だったか/チームとして志向するスタイル等が見えるデータを提供している。

### 提供しているデータ

提供されているデータの中から、チーム毎のデータとして、シーズン毎の
* Team Data
  * 順位表
  * チャンス構築率
  * 得点パターン
  * 失点パターン
  * ボール支配率
* チームスタイル指標
  * ショートカウンター
  * ロングカウンター
  * 敵陣ポゼッション
  * 自陣ポゼッション
  * 左サイド攻撃
  * 中央攻撃
  * 右サイド攻撃
* チャンスビルディングポイント
  * 攻撃
  * パス
  * ドリブル
  * クロス
  * シュート
  * 守備
  * セーブ

を収集してみる。

### Team Data: 順位表

現時点の順位表データを取得する。

In [8]:
import pandas as pd
def get_rank_table(competition=1, year=2018):
    url = "http://www.football-lab.jp/summary/team_ranking/j" + str(competition) + "/?year=" + str(year)
    dfs = pd.io.html.read_html(url)
    df = dfs[0].drop("Unnamed: 1",axis=1).rename(columns={"Unnamed: 2":"チーム"})
    return df

In [9]:
rank = get_rank_table(competition=1, year=2017)
#display(rank)

チーム名がおかしいが、省略しないチーム名+省略名(公式データの省略名と同じ)になっているようなので、
公式データのチーム名と比較することにする。

In [10]:
import download
df = download.get_db_data()

In [11]:
teams = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates()
#display(teams)

In [12]:
for i in range(len(rank["チーム"])):
    for team in teams:
        if rank.loc[i, "チーム"].endswith(team) is True:
            rank.loc[i, "チーム"] = team
            print(rank["チーム"].iloc[i])

川崎Ｆ
鹿島
Ｃ大阪
柏
横浜FM
磐田
浦和
鳥栖
神戸
Ｇ大阪
札幌
仙台
FC東京
清水
広島
甲府
新潟
大宮


In [13]:
import download
rank = download.fl_get_rank(1,2018)
#display(rank)

### Team Data:チャンス構築率

|データ項目|プレー定義|
|----------|----------|
|攻撃(回数)|ある特定の状況において例外はあるものの、ボールを保持してから相手チームに渡る、<br>もしくはファウルやボールアウトで試合が止まるまでの間を１回の攻撃とする|
|シュート|データスタジアムが独自に集計したシュート数であり、公式記録の数値とは異なる|
|チャンス構築率|上記のシュート数÷攻撃回数により算出|
|ゴール|得点となったシュートの数。オウンゴールは含まれない|
|シュート成功率|上記のゴール数÷シュート数により算出|

In [14]:
url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2018&data=chance"
dfs = pd.io.html.read_html(url)
#print(dfs)

In [15]:
def get_chance_rate(competition=1, year=2018):
    url = "http://www.football-lab.jp/summary/team_ranking/j" + str(competition) + "/?year=" + str(year) + "&data=chance"
    dfs = pd.io.html.read_html(url)
    attack = dfs[0].drop("Unnamed: 0",axis=1)
    attack = attack.drop("Unnamed: 3",axis=1)
    attack = attack.drop("Unnamed: 5",axis=1)
    attack = attack.drop("Unnamed: 7",axis=1)
    attack = attack.drop("Unnamed: 9",axis=1)
    attack = attack.drop("Unnamed: 11",axis=1)
    attack = attack.rename(columns={"Unnamed: 1":"チーム"})
    defense = dfs[2].drop("Unnamed: 0",axis=1)
    defense = defense.drop("Unnamed: 3",axis=1)
    defense = defense.drop("Unnamed: 5",axis=1)
    defense = defense.drop("Unnamed: 7",axis=1)
    defense = defense.drop("Unnamed: 9",axis=1)
    defense = defense.drop("Unnamed: 11",axis=1)
    defense = defense.rename(columns={"Unnamed: 1":"チーム"})
    return attack, defense

In [16]:
atk, dfe = get_chance_rate()
#display(atk)

In [17]:
atk, dfe = download.fl_get_chance_rate(competition=1,year=2018)
#display(atk)

### Team Data: ゴールパターン&失点パターン

テーブル属性ではないので、Beautifulsoupで収集。

In [18]:
from pprint import pprint
import requests
import re
import ast
from bs4 import BeautifulSoup,Comment

url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2018&data=goal"

response = requests.get(url)
bs = BeautifulSoup(response.content,"lxml")
test=str(bs.find(string=re.compile("function drawChart")))

tbl = re.search(r'arrayToDataTable\((.*?)\)', test, flags=re.DOTALL|re.MULTILINE).group(1)
pat = pd.DataFrame(ast.literal_eval(tbl)[1:],columns=ast.literal_eval(tbl)[0])
#display(pat)

In [35]:
pat = download.fl_get_goal_pattern(competition=2, year=2018)
display(pat)

,チーム,ＰＫ,セットプレー直接,セットプレーから,クロスから,スルーパスから,ショートパスから,ロングパスから,ドリブルから,こぼれ球から,その他
0,山口,2,0,9,11,2,10,3,4,6,4
1,大宮,5,5,8,8,5,10,2,6,0,2
2,千葉,5,1,14,14,1,6,0,2,6,2
3,大分,3,1,5,12,6,8,2,2,4,7
4,町田,1,1,19,7,3,9,0,2,1,6
5,甲府,3,0,12,5,4,6,0,7,4,5
6,横浜FC,3,2,12,13,1,10,0,0,2,2
7,松本,4,1,14,4,1,5,3,5,3,3
8,東京Ｖ,1,0,12,9,5,2,5,3,3,2
9,福岡,3,3,10,2,2,9,1,4,1,5


### Team Data: ボール支配率


In [20]:
url = "http://www.football-lab.jp/summary/team_ranking/j1/?year=2017&data=possession"
response = requests.get(url)
bs = BeautifulSoup(response.content,"lxml")
test=str(bs.find(string=re.compile("function drawChart"))).replace("(%)","%")    # なぜか(%)のところが変換できないので文字列として変換
tbl = re.search(r'arrayToDataTable\((.*?)\)', test, flags=re.DOTALL|re.MULTILINE).group(1)
tbl = re.sub(", \{.+?\}","", tbl)   # annotation属性が残ってしまうので削除
col = ast.literal_eval(tbl)[0]
col.append("dummy")
pos = pd.DataFrame(ast.literal_eval(tbl)[1:],columns=col).drop("dummy",axis=1) # 不必要な列があるのでdummyと命名して削除
#display(pos)

In [21]:
pos = download.fl_get_possession(competition=2,year=2018)
#display(pos)

### チャンスビルディングポイント

URLは`http://www.football-lab.jp/summary/cbp_ranking/j1/?year=<YEAR>&data=<DATA>`」で、

|データ|<DATA>|
|-----|-------|
|攻撃|なし|
|パス|pass|
|ドリブル|dribble|
|クロス|cross|
|シュート|shot|
|守備|defense|
|セーブ|save|

という差。
* チーム名がUnnamed: 2になっていて、Unnnamed: 0-1は不要。
* 順位表同様、チーム名はリネームする。
* 順位・勝ち点・得点・失点は別のデータにあるので削除しておく

In [22]:
dfs = pd.io.html.read_html("http://www.football-lab.jp/summary/cbp_ranking/j1/?year=2013")
#display(dfs[0])

In [25]:
cbp = download.fl_get_cbp(competition=2, year=2017, data="dribble")
#display(cbp)

### まとめ

ここまでのデータを、チーム名でまとめてデータフレーム化する。

得点パターンと失点パターンが一緒なので、renameする。

In [26]:
rank = download.fl_get_rank(1,2018)
atk, dfe = download.fl_get_chance_rate(competition=1,year=2018)
goal_pat = download.fl_get_goal_pattern(competition=1, year=2018)
goal_pat = goal_pat.rename(columns={"ＰＫ":"ＰＫ_得点",
                                    "セットプレー直接":"セットプレー直接_得点",
                                    "セットプレーから":"セットプレーから_得点",
                                    "クロスから":"クロスから_得点",
                                    "スルーパスから":"スルーパスから_得点",
                                    "ショートパスから":"ショートパスから_得点",
                                    "ロングパスから":"ロングパスから_得点",
                                    "ドリブルから":"ドリブルから_得点",
                                    "こぼれ球から":"こぼれ球から_得点",
                                    "その他":"その他_得点"
                                   })
lost_pat = download.fl_get_lost_pattern(competition=1, year=2018)
lost_pat = lost_pat.rename(columns={"ＰＫ":"ＰＫ_失点",
                                    "セットプレー直接":"セットプレー直接_失点",
                                    "セットプレーから":"セットプレーから_失点",
                                    "クロスから":"クロスから_失点",
                                    "スルーパスから":"スルーパスから_失点",
                                    "ショートパスから":"ショートパスから_失点",
                                    "ロングパスから":"ロングパスから_失点",
                                    "ドリブルから":"ドリブルから_失点",
                                    "こぼれ球から":"こぼれ球から_失点",
                                    "その他":"その他_失点"
                                   })
posession = download.fl_get_possession(competition=1,year=2018)

#fl_data = pd.merge(rank, atk, dfe, goal_pat, lost_pat, posession)
fl_data = pd.merge(rank, atk, on="チーム")
fl_data = pd.merge(fl_data, dfe, on="チーム")
fl_data = pd.merge(fl_data, goal_pat, on="チーム")
fl_data = pd.merge(fl_data, lost_pat, on="チーム")
fl_data = pd.merge(fl_data, posession, on="チーム")
fl_data["年度"] = 2018
fl_data["ディビジョン"] = "J" + str(1)
#display(fl_data)

## 使い方

``` python
download.download_all_fl_data()
```
を実施すると、DBのdataテーブルに保存される。

実施後は、
``` python
data = download.get_db_data(data="data")
```
で取得できる。

In [1]:
import download
data = download.download_all_fl_data()
#display(data)

In [32]:
data = download.get_db_data(data="data")
display(data)

DatabaseError: Execution failed on sql 'select * from data': no such table: data